In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
os.environ["MALLOC_TRIM_THRESHOLD_"] = '0'

import numpy as np
import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

from functionMaths import *

plt.rc("figure", figsize=(12,10))
plt.rc("font", size=14)

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, protocol = 'tcp', host = '137.129.155.67')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37935 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:37935/status,
Dashboard: http://137.129.155.67:37935/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://137.129.155.67:36729,Workers: 1
Dashboard: http://137.129.155.67:37935/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://137.129.155.67:45559,Total threads: 4
Dashboard: http://137.129.155.67:36947/status,Memory: 78.61 GiB
Nanny: None,


In [8]:
np.linspace(0, , 8, dtype = 'int')


array([ 0,  4,  8, 13, 17, 22, 26, 31])

In [2]:
wave_OLR = ['OLR_TD', 'OLR_MJO', 'OLR_Kelvin', 'OLR_Rossby', 'OLR_MRG', 'OLR_EIG','OLR_WIG']
indir_data = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/'
coeff = 1


In [3]:
ds = xr.open_mfdataset(indir_data + '*.nc', chunks = {'time' : 10}, parallel = True)
_ds = ds.sel(lat = slice(-19,19))
_ds = _ds.sel(lon = 140.5).persist()

### Compute temporal derivative
_ds_DDt = _ds.differentiate('time', datetime_unit = "s")

ds_std = _ds.std(dim = ['time','lat'])
ds_DDt_std = _ds_DDt.std(dim = ['time','lat'])
# Normalizing by standard deviation
ds_norm = _ds/ds_std
_ds_DDt = _ds_DDt/ds_DDt_std

### Stock in one datasets
for w in wave_OLR :
    ds_norm[w + '_DT'] = _ds_DDt[w]
    
### Compute variance of the data
ds_norm_std = ds_norm.std(dim = 'time')

v1 = ds_norm['OLR_Kelvin'].load()
v2 = ds_norm['OLR_Kelvin_DT'].load()

ds_plot = amplitudeMagnitude(v1, v2)

space_phase = np.linspace(np.pi, -np.pi, 33, endpoint = True)
d_space_phase = (space_phase[1] - space_phase[2])/2
space_phase = space_phase[:-1] - d_space_phase

nb_phase = np.linspace(0, 31, 32, dtype = 'int')

ds_plot = ds_plot.expand_dims(phase = nb_phase, axis = 2)

ampl = np.arctan2(v2,v1)  ### arctan2 variable on y first, on x second
# magn = np.sqrt(v1**2 + v2**2)

# ampl = ampl.rename('ampl')
# magn = magn.rename('magn')
# __ds['magnitude'] = magn

# # std_circle = magn.std('time')
# # std_circle = (magn * 0. + std_circle).persist()
# # magn = magn.where(magn >= std_circle)

__ds = ds_plot #.transpose("time","lat","lon","phase")

for i in range(nb_phase.size-1) :
    da = xr.where( ((ampl <=  space_phase[i]) & ( ampl > space_phase[i+1]) == True),
                                       ampl, np.nan)
    __ds['amplitude'][...,i+1] = da
    del da
    
da = xr.where( ((ampl <=  space_phase[-1]) | ( ampl > space_phase[0]) == True),
                                       ampl, np.nan)
__ds['amplitude'][...,0] = da 

# __ds['magnitude'] = __ds['magnitude'].expand_dims(phase = __ds['phase'], axis = 2)
__ds['magnitude'] = xr.where(__ds['amplitude'].isnull() != True, __ds['magnitude'], np.nan)

__ds = __ds.mean(dim = 'lon')
__ds['vect_x'] = __ds['magnitude'] * np.cos(__ds['amplitude'])
__ds['vect_y'] = __ds['magnitude'] * np.sin(__ds['amplitude'])

ValueError: Assignment destination is a view.  Do you want to .copy() array first?

In [ ]:
_ds

In [ ]:
_ds_2 = __ds.expand_dims(phase = __ds['phase'], axis = 2)
_ds_2 = xr.where(__ds['amplitude'].isnull() != True, __ds_2, np.nan)
__da = __ds_2['OLR_Kelvin'].mean(dim = 'time').compute()
__da.plot.contourf()

In [ ]:
__ds_mean = __ds.mean(dim = 'time').compute()
__ds_mean['magnitude'].plot.contourf()

In [ ]:
fig, axes = plt.subplots(figsize=(15,15))
# _ds_FILTER_mean = ds_FILTER_mean.sel(level = 850)
variable = ['MJO','TD','Kelvin','Rossby','MRG']
plt.rc("figure", figsize=(12,10))
# ds_norm.sel(lat = 10, method = 'nearest').plot.scatter(x = 'OLR_Kelvin', y = 'OLR_' + _wave + '_DT')
__ds.sel( phase = 0, lat = 10, method = 'nearest').plot.scatter(x = 'vect_x', y = 'vect_y')
axes.set_ylabel('D(OLR)/Dt ')
axes.set_label('anomalie totale OLR')
axes.hlines(0, -250, 100, colors = 'black')
axes.vlines(0, -55, 55, colors = 'black')
axes.set_title('anomlie totale OLR')
axes.set_xlim(-4.1,4.1)
axes.set_ylim(-4.1,4.1)
axes.grid()

In [ ]:
___ds = __ds.mean('time')
___ds

In [ ]:
ampl = np.arctan2(v2,v1)  ### arctan2 variable on y first, on x second
magn = np.sqrt(v1**2 + v2**2)


std_circle = magn.std('time')
std_circle = magn.copy() * 0. + std_circle

ampl = ampl.rename('amplitude')
magn = magn.rename('magnitude')
std_circle = std_circle.rename('std')
# Create new dataset to sctock all the variable
_ds = xr.merge([ampl, magn, std_circle]) 

_ds['vect_x'] = magn*np.cos(ampl)
_ds['vect_y'] = magn*np.sin(ampl)

_ds['std_circle_x'] = std_circle*np.cos(ampl)
_ds['std_circle_y'] = std_circle*np.sin(ampl)

_ds

In [ ]:
ds_plot = amplitudeMagnitude(v1, v2)
ds_plot

In [ ]:
da = xr.DataArray(
    data=numpy_data,
    dims=["time", "phase"],
    coords=dict(
        time = ("time", ds_plot.time.data),
        phase = ("phase", nb_phase),
    ),
    attrs=dict(
        description="phase of the wave",
        units="int",
    ),
)
ds_plot['sector'] = da
ds_plot

In [ ]:
ds_plot.coords['phase'] = nb_phase
ds_plot
k = 0
for i in range(0,31):
    ds_plot['sector'][:,i] = xr.where( ((ds_plot.amplitude)>space_phase[i]) & 
                                            ((ds_plot.amplitude)<=space_phase[i+1]) == True, 
                                      ds_plot.amplitude,0)


In [ ]:
ds_plot['sector'].sel(time=slice('2009-07-01','2009-07-10'),phase = 12).plot()

In [ ]:
_wave = 'Kelvin'

fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(10,15))
# _ds_FILTER_mean = ds_FILTER_mean.sel(level = 850)
variable = ['MJO','TD','Kelvin','Rossby','MRG']
plt.rc("figure", figsize=(12,10))
ds_norm.plot.scatter(ax=axes[0, 0], x = 'OLR_ano', y = 'OLR_' + _wave + '_DT')
axes[0,0].set_ylabel('D(OLR)/Dt ' + _wave)
axes[0,0].set_label('anomalie totale OLR')
axes[0,0].hlines(0, -250, 100, colors = 'black')
axes[0,0].vlines(0, -55, 55, colors = 'black')
axes[0,0].set_title('anomlie totale OLR')
axes[0,0].set_xlim(-4.1,4.1)
axes[0,0].set_ylim(-4.1,4.1)
axes[0,0].grid()
# plt.show()

i = 1
j = 0
for w in wave_OLR :
    ds_norm.plot.scatter(ax=axes[j, i], x = w, y = w + '_DT')
    axes[j,i].set_ylabel('D(OLR)/Dt ' + w)
    axes[j,i].set_xlabel('anomalie '+ w)
    axes[j,i].hlines(0, -250, 100, colors = 'black')
    axes[j,i].vlines(0, -55, 55, colors = 'black')
    axes[j,i].set_title(w )
    axes[j,i].set_xlim(-4.1,4.1)
    axes[j,i].set_ylim(-4.1,4.1)
    axes[j,i].grid()
    i = i + 1
    if i%2 == 0:
        i = 0
        j = j + 1
        
fig.tight_layout(pad=3.0)
plt.suptitle('scatter plot entre l OLR' + _wave )

In [ ]:
ds_norm.plot.scatter(x = 'OLR_TD', y = 'OLR_TD_DT')